In [33]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

import  yfinance as yf

In [34]:
# Get historical OHLCV data for target ticker
etf_df = yf.download(
    "STW.AX, VAS.AX, SPY.AX", 
    period="max"
)

[*********************100%***********************]  3 of 3 completed


In [35]:
# Review data to see the date ranges
display(etf_df.head())
display(etf_df.tail())
display(etf_df.shape)
display(etf_df.index)


Adj Close                    Close                     High  \
              SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX   
Date                                                                     
2008-01-02       NaN  61.169998    NaN    NaN  61.169998    NaN    NaN   
2008-01-03       NaN  60.590000    NaN    NaN  60.590000    NaN    NaN   
2008-01-04       NaN  60.740002    NaN    NaN  60.740002    NaN    NaN   
2008-01-07       NaN  59.349998    NaN    NaN  59.349998    NaN    NaN   
2008-01-08       NaN  59.020000    NaN    NaN  59.020000    NaN    NaN   

                                Low                     Open             \
               STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX   
Date                                                                      
2008-01-02  61.169998    NaN    NaN  61.169998    NaN    NaN  61.169998   
2008-01-03  60.590000    NaN    NaN  60.590000    NaN    NaN  60.590000   
2008-01-04  60.740002    NaN    NaN  60.740002    NaN    NaN  60.740002   
2008-01-07  59.349998    NaN    NaN  59.349998    NaN    NaN  59.349998   
2008-01-08  59.020000    NaN    NaN  59.020000    NaN    NaN  59.020000   

                  Volume                
           VAS.AX SPY.AX STW.AX VAS.AX  
Date                                    
2008-01-02    NaN    NaN      0    NaN  
2008-01-03    NaN    NaN      0    NaN  
2008-01-04    NaN    NaN      0    NaN  
2008-01-07    NaN    NaN      0    NaN  
2008-01-08    NaN    NaN      0    NaN

Adj Close                             Close             \
                SPY.AX     STW.AX     VAS.AX      SPY.AX     STW.AX   
Date                                                                  
2022-12-16  578.650024  65.010002  88.980003  578.650024  65.010002   
2022-12-19  571.799988  64.849998  88.800003  571.799988  64.849998   
2022-12-20  567.669983  63.900002  87.440002  567.669983  63.900002   
2022-12-21  573.729980  64.669998  88.500000  573.729980  64.669998   
2022-12-22  573.000000  65.040001  88.949997  573.000000  65.040001   

                             High                               Low  \
               VAS.AX      SPY.AX     STW.AX     VAS.AX      SPY.AX   
Date                                                                  
2022-12-16  88.980003  583.880005  65.339996  89.440002  571.969971   
2022-12-19  88.800003  574.559998  65.040001  89.050003  571.500000   
2022-12-20  87.440002  569.679993  64.809998  88.709999  562.400024   
2022-12-21  88.500000  574.520020  64.980003  88.839996  570.059998   
2022-12-22  88.949997  578.000000  65.199997  89.180000  573.000000   

                                        Open                       Volume  \
               STW.AX     VAS.AX      SPY.AX     STW.AX     VAS.AX SPY.AX   
Date                                                                        
2022-12-16  64.730003  88.570000  583.880005  64.879997  88.800003  199.0   
2022-12-19  64.709999  88.570000  574.559998  64.730003  88.589996  453.0   
2022-12-20  63.799999  87.300003  567.750000  64.809998  88.570000  779.0   
2022-12-21  64.430000  88.129997  570.059998  64.430000  88.139999  435.0   
2022-12-22  65.019997  88.879997  577.330017  65.110001  89.000000  199.0   

                               
            STW.AX     VAS.AX  
Date                           
2022-12-16  211859   155356.0  
2022-12-19   65356   239837.0  
2022-12-20  125904   357111.0  
2022-12-21   69366   154546.0  
2022-12-22   52931  1206436.0

(3790, 18)

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2022-12-09', '2022-12-12', '2022-12-13', '2022-12-14',
               '2022-12-15', '2022-12-16', '2022-12-19', '2022-12-20',
               '2022-12-21', '2022-12-22'],
              dtype='datetime64[ns]', name='Date', length=3790, freq=None)

In [36]:
# Examine the column type and check for nulls
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3790 entries, 2008-01-02 to 2022-12-22
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   (Adj Close, SPY.AX)  2074 non-null   float64
 1   (Adj Close, STW.AX)  3790 non-null   float64
 2   (Adj Close, VAS.AX)  3453 non-null   float64
 3   (Close, SPY.AX)      2074 non-null   float64
 4   (Close, STW.AX)      3790 non-null   float64
 5   (Close, VAS.AX)      3453 non-null   float64
 6   (High, SPY.AX)       2074 non-null   float64
 7   (High, STW.AX)       3790 non-null   float64
 8   (High, VAS.AX)       3453 non-null   float64
 9   (Low, SPY.AX)        2074 non-null   float64
 10  (Low, STW.AX)        3790 non-null   float64
 11  (Low, VAS.AX)        3453 non-null   float64
 12  (Open, SPY.AX)       2074 non-null   float64
 13  (Open, STW.AX)       3790 non-null   float64
 14  (Open, VAS.AX)       3453 non-null   float64
 15  (Volume, SPY.AX)    

In [37]:
# Retain the key columns for each etf
etf_df = etf_df.drop(columns = ['Adj Close', 'High', 'Low', 'Open', 'Volume'])
etf_df.columns = etf_df.columns.droplevel()     # Changes the multilevel indexing on columns to single level


In [38]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2008-01-02,NaN,61.169998,NaN
2008-01-03,NaN,60.590000,NaN
2008-01-04,NaN,60.740002,NaN
2008-01-07,NaN,59.349998,NaN
2008-01-08,NaN,59.020000,NaN


In [39]:
#  Drop all nulls
etf_df = etf_df.dropna()
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2073 entries, 2014-10-13 to 2022-12-22
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SPY.AX  2073 non-null   float64
 1   STW.AX  2073 non-null   float64
 2   VAS.AX  2073 non-null   float64
dtypes: float64(3)
memory usage: 64.8 KB


In [40]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2014-10-13,216.729996,48.599998,65.360001
2014-10-14,213.949997,49.119999,66.059998
2014-10-15,217.089996,49.500000,66.480003
2014-10-16,212.434738,49.549999,66.599998
2014-10-17,213.059998,49.779999,66.839996
